# 第7回 その2: 畳み込みと残響
ここでは畳み込みの実装コードを紹介し，音声とインパルス応答の畳み込みによる残響音声のデモンストレーションを行います。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/07"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/07/
# 現在のフォルダの中身を表示
%ls

`input.wav`，`rir_310.wav`，`rir_1000.wav`，`rir_hall.wav`というデータが表示されていることを確認してください。

必要なライブラリをインポートしておきます。

In [ ]:
import wave as wave
import numpy as np
import matplotlib.pyplot as plt

## ステップ1: 畳み込み
畳み込みは以下の式で定義されます。  
$z[n] = \sum_{k=0}^{K-1}h[k]x[n-k]$  
$n$，$k$ はそれぞれ時刻を表す記号です。  
$x[n]$ は畳み込まれる前の源信号です。  
$h[k]$ は畳み込む信号（インパルス応答）で，$k=0,\dots,K-1$です。  
$z[n]$ は $x$ に $h$ が畳み込まれた結果の信号です。  

以下に $x$ と $h$ の畳み込みを行う関数を定義します。

In [ ]:
def convolve(x, h):
  N = np.size(x) # ベクトル x の長さ
  K = np.size(h) # ベクトル h の長さ
  z = np.zeros(N) # 畳み込み結果のベクトル（長さN）
  
  for n in range(N):
    if n-K+1 < 0:
      # x[n-k+1]のインデクスが0以下になる場合は畳み込みをしない。
      z[n] = x[n]
    else:
      # nからK個分前の要素(n-K+1)からn番目までの要素(n+1)を取り出す
      x_part = x[n-K+1 : n+1]
      # 取り出したベクトルを逆順に並び替える
      x_rev = np.flip(x_part)
      # 逆順に並び替えたベクトルと h の内積を計算
      z[n] = np.dot(h, x_rev)
  
  return z

定義した関数を使って，$x = [0, 1, 2, 3, 4, 5]$，$h = [0.2, 0.3, 0.4]$として畳み込みを実行してみます。  

In [ ]:
x = np.array([0,1,2,3,4,5])
h = np.array([0.2, 0.3, 0.4])
# 畳み込み実行
z = convolve(x, h)
print('z = x*h = ' + str(z))

$z = [0.0, 1.0, 0.7, 1.6, 2.5, 3.4]$となります。  

[補足]  
最初の２要素（$0.0, 1.0$）は $n-K+1 < 0$ となるため，畳み込みを実行していません。実はこのエラー処理はかなり雑で，本来は $n-K+1 < 0$ となる場合は $K$ を $K - n-1$に置き換えるなどの処理をするのが正しいです。ですが，ここでは簡単な実装を採用しています。

## ステップ2: インパルス応答の畳み込みと残響  
上で定義した畳み込み関数を使って，実際に音声とインパルス応答を畳み込んでみましょう。  
まずは音声データ`input.wav`を読み込みます。

In [ ]:
wav_file = 'input.wav'
with wave.open(wav_file, 'rb') as wav:
  # サンプリング周波数 [Hz] を取得
  sampling_frequency = wav.getframerate()
  # その他の情報（ファイルサイズ等，書き込み時に必要）を取得
  wav_params = wav.getparams()
  # wavデータを読み込む
  input_data = wav.readframes(wav.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  input_data = np.frombuffer(input_data, dtype=np.int16).astype(np.float32)

# データをプロット
plt.figure(figsize=(10,5))
x = np.arange(np.size(input_data)) / sampling_frequency # 横軸データ
plt.plot(x, input_data)
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.show()

続いてインパルス応答データ `rir_310.wav` も読み込みます。  
（インパルス応答は音声ではありませんが，ここでは処理を簡単にするため，音声と同じwav形式のファイルで作成しています。）  
また，インパルス応答は最大値が1.0になるように正規化しておきます。

In [ ]:
rir_file = 'rir_310.wav'
with wave.open(rir_file, 'rb') as rir:
  # （サンプリング周波数やその他の情報の取得は省略する。）
  # wavデータを読み込む
  rir_data = rir.readframes(rir.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  rir_data = np.frombuffer(rir_data, dtype=np.int16).astype(np.float32)

  # 最大値が1.0になるように正規化
  rir_data = 1.0 * rir_data / 32768

# データをプロット
plt.figure(figsize=(10,5))
x_r = np.arange(np.size(rir_data)) / sampling_frequency # 横軸データ
plt.plot(x_r, rir_data)
plt.xlabel('Time [second]')
plt.ylabel('Impulse response')
plt.show()

読み込んだ音声データ `input_data` と インパルス応答 `rir_data` を畳み込みます。

In [ ]:
# インパルス応答との畳み込みを実行
output_data = convolve(input_data, rir_data)

# データをプロット
plt.figure(figsize=(10,8))
plt.subplot(2, 1, 1)
plt.plot(x, input_data, label='input')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(x, output_data, label='output')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

畳み込んだ結果を`output.wav`というファイル名で書き込み，元の音声信号と聞き比べてみましょう。  
書き込むとき，`output_data` は float型になっているので，  
`output_data = output_data.astype(np.int16)`  
として short 型に変更してから`tobytes`でバイナリデータに変換している点に注意してください。  
（これをしないと出力したwavデータはめちゃくちゃな音になります。）

In [ ]:
out_file = 'output.wav'
with wave.open(out_file, 'wb') as out:
  # 音声データの情報（wav_params）をセット
  out.setparams(wav_params)
  # numpy array型(32bit float)のデータをバイナリデータ（16bit short）に変換
  out_binary_data = output_data.astype(np.int16).tobytes()
  # データを書き込む
  out.writeframes(out_binary_data)

import IPython.display
print('input.wav')
IPython.display.display(IPython.display.Audio('input.wav'))
print('output.wav')
IPython.display.display(IPython.display.Audio('output.wav'))

両者を聞き比べると，畳み込んだ音声はややくぐもった音声に聞こえるのが確認できるはずです。  
このように，部屋の反射によって過去の音声が畳み込まれる現象のことを<font color='red'>**残響**</font>と呼びます。

## ステップ3: インパルス応答の違いによる残響音声の変化  
他のインパルス応答`rir_1000.wav`や`rir_hall.wav`での畳み込みも試してみましょう。  
以下は，上記のソースコードを `rir_file`のファイル名を変えてコピペしているだけです。 

In [ ]:
rir_file = 'rir_1000.wav'
with wave.open(rir_file, 'rb') as rir:
  # （サンプリング周波数やその他の情報の取得は省略する。）
  # wavデータを読み込む
  rir_data = rir.readframes(rir.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  rir_data = np.frombuffer(rir_data, dtype=np.int16).astype(np.float32)

  # 最大値が1.0になるように正規化
  rir_data = 1.0 * rir_data / 32768

# データをプロット
plt.figure(figsize=(10,5))
x_r = np.arange(np.size(rir_data)) / sampling_frequency # 横軸データ
plt.plot(x_r, rir_data)
plt.xlabel('Time [second]')
plt.ylabel('Impulse response')
plt.show()

# インパルス応答との畳み込みを実行
output_data = convolve(input_data, rir_data)

# データをプロット
plt.figure(figsize=(10,8))
plt.subplot(2, 1, 1)
plt.plot(x, input_data, label='input')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(x, output_data, label='output')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

out_file = 'output.wav'
with wave.open(out_file, 'wb') as out:
  # 音声データの情報（wav_params）をセット
  out.setparams(wav_params)
  # numpy array型(32bit float)のデータをバイナリデータ（16bit short）に変換
  out_binary_data = output_data.astype(np.int16).tobytes()
  # データを書き込む
  out.writeframes(out_binary_data)

import IPython.display
print('input.wav')
IPython.display.display(IPython.display.Audio('input.wav'))
print('output.wav')
IPython.display.display(IPython.display.Audio('output.wav'))

In [ ]:
rir_file = 'rir_hall.wav'
with wave.open(rir_file, 'rb') as rir:
  # （サンプリング周波数やその他の情報の取得は省略する。）
  # wavデータを読み込む
  rir_data = rir.readframes(rir.getnframes())
  # 読み込んだデータはバイナリ値(16bit short型)なので，numpy array型の数値ベクトル(32bit float型)に変換する
  rir_data = np.frombuffer(rir_data, dtype=np.int16).astype(np.float32)

  # 最大値が1.0になるように正規化
  rir_data = 1.0 * rir_data / 32768

# データをプロット
plt.figure(figsize=(10,5))
x_r = np.arange(np.size(rir_data)) / sampling_frequency # 横軸データ
plt.plot(x_r, rir_data)
plt.xlabel('Time [second]')
plt.ylabel('Impulse response')
plt.show()

# インパルス応答との畳み込みを実行
output_data = convolve(input_data, rir_data)

# データをプロット
plt.figure(figsize=(10,8))
plt.subplot(2, 1, 1)
plt.plot(x, input_data, label='input')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.subplot(2, 1, 2)
plt.plot(x, output_data, label='output')
plt.xlabel('Time [second]')
plt.ylabel('Amplitude')
plt.legend()
plt.show()

out_file = 'output.wav'
with wave.open(out_file, 'wb') as out:
  # 音声データの情報（wav_params）をセット
  out.setparams(wav_params)
  # numpy array型(32bit float)のデータをバイナリデータ（16bit short）に変換
  out_binary_data = output_data.astype(np.int16).tobytes()
  # データを書き込む
  out.writeframes(out_binary_data)

import IPython.display
print('input.wav')
IPython.display.display(IPython.display.Audio('input.wav'))
print('output.wav')
IPython.display.display(IPython.display.Audio('output.wav'))

インパルス応答が長くなるにしたがって，残響の影響も大きくなることが確認できたはずです。  
ちなみに，`rir_hall.wav`はコンサートホールで収録されたインパルス応答です。  
インパルス応答を事前に計測して記録しておくことで，その場で収録したような音声がシミュレーション生成できるということです。